**import nessecary modules**

In [ ]:
from bs4 import BeautifulSoup 
import requests
import re
import pandas as pd

**connect to api**

In [ ]:
url = f'https://api.genius.com/oauth/authorize?client_id={Client_ID}&scope=create_annotation&response_type=code'
requests.get(url)

In [ ]:
#my private api
App_Website_URL = 'https://pf.hse.ru/909509815.html'
Client_ID = 'SqR_LrstmrTJx-Y-nQtDB3sTakUnAzGNhFjy1BrfPnOPctH-yXIEdEbIbE0XN2tF'
Client_Secret = '6JQD5TnnO74NWKSMtbn-s1vCghAMCteVWzaLPFK9lI4h-tJclLkrvo96QoV2MBrFTvoFbZ6pV6rnbf78EwBYpw'
Client_Access_Token = '-RZuj6F8pe6R7Tg7_bwy3tmISIDLnos7FOj5-ACIT6kER3nzokeHOOEPS_UWScDs'


**collect an artist id by name**

in case of we can search an information about artist only by hidden id, we need to cellect it first

In [ ]:
def get_artist_id(artist_name):
    url = 'https://api.genius.com/search'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    params = {'q': artist_name}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        artist_id = data['response']['hits'][0]['result']['primary_artist']['id']
        return artist_id
    else:
        print('Error:', response.status_code)
        return None

artist_id = get_artist_id('Night-snipers')

**request a song links from id**

genius can give only 50 links per page, so we use *for* construction to collect all links. Surpriselly, genius don't send an error if our page out of range, so we can put random page number

In [ ]:
GENIUS_API_TOKEN = 'kayRq4mQVfs7tlzkcDADdLW-0sp4Pf-N5I3b5PbMGTJz-qHGRt8jrSDEielXXBIu'

urls = []

for page in range(3):
    try:
        url = f'https://api.genius.com/artists/{artist_id}/songs?per_page=50&page={page}'
        headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
        
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            response = response.json()
            urls_temp = [i['path'] for i in response['response']['songs']]
            urls.extend(urls_temp)
        else:
            print('Error:', response.status_code)
    except Exception as e:
        print(':(', e)

print(urls)


our *url* is in short form, so to use it in the future, we need to transform to normal url:

In [ ]:
full_url = ['https://genius.com' + i for i in urls]

**Let's collect an lyrics from links**

Genius API isn't convenient to scrap a lyrics in case of thay json answer and scop setting. Anyway, every genius page has the similar strucrure, so we can easy process them with BeautifulSoup

In [ ]:
full_df = []
def texter(links):
    text = []
    page = requests.get(links)
    soup = BeautifulSoup(page.text)
    try:
        lyrics_div = soup.find_all('div', {'class':'Lyrics__Container-sc-1ynbvzw-1 kUgSbL'})[0]
    except:
        lyrics_div = soup.find('div', {'class':'Lyrics__Container-sc-1ynbvzw-1 kUgSbL'})
    if lyrics_div:
        lyrics_text = lyrics_div.get_text(separator='\n')
        lyrics_lines = [line.strip() for line in lyrics_text.split('\n') if line.strip()]
        cleaned_lyrics = []
        for line in lyrics_lines:
            cleaned_line = re.sub(r'\[.*?\]', '', line)
            cleaned_lyrics.append(cleaned_line.strip())
        for line in cleaned_lyrics:
            if line != '':
                text.append(line)
    text = ' '.join(text)
    return text

for link in full_url:
    result = texter(link)
    print(link)
    full_df.append(result)

full_df is a list of lists, so we need to make one joined text in main_df:

In [ ]:
main_df = []
for i in full_df:
    if i != '':
        main_df.append(''.join(i))
